In [68]:
import pandas as pd
import numpy as np
from lifelines import CoxPHFitter
import os
from sklearn.model_selection import train_test_split
from scipy.stats import entropy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sksurv.datasets import load_breast_cancer
from sksurv.linear_model import CoxPHSurvivalAnalysis, CoxnetSurvivalAnalysis
from sksurv.preprocessing import OneHotEncoder
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import pickle
from datetime import datetime
from numbers import Number

In [74]:
cancers = ['BLCA', 'BRCA', 'HNSC', 'LAML', 'LGG', 'LUAD']
path_prefix = 'C:/Users/sharony/SurvivalAnalysis/'
if os.getlogin() =='meiry':
    path_prefix = 'D:/sharon/medical_genomics_data/'

min_feature_std = 0.00 #0.01
# Drop features with low std 

In [204]:

def calc_time_to_event(x):
    #print(type(x) , x)
    assert x.vital_status in ['Dead', 'Alive']
    if x.vital_status=='Dead':
        try:
            assert isinstance(int(x.death_days_to), int)
        except:
            print(type(x.death_days_to), 'non int entry in x.death_days_to' , x.death_days_to, 'removing row',x.vital_status, x.death_days_to)        
            return None
        assert float(x.death_days_to) >= 0
        return int(x.death_days_to)
    try:
        assert isinstance(int(x.last_contact_days_to), int) 
    except :
        print(type(x.last_contact_days_to), 'non int entry in x.last_contact_days_to' , x.last_contact_days_to, 'removing row',x.vital_status, x.death_days_to)        
        return None
    if int(x.last_contact_days_to) < 0:
        print('negative entry in x.last_contact_days_to' , x.last_contact_days_to, 'fixing it')
    return abs(int(x.last_contact_days_to))


In [77]:
def correlated_features(ct, comment, omics, min_correlation_threshold = 0.9, max_features_to_correlate = None  ):
    if max_features_to_correlate is None: 
        max_features_to_correlate = omics.shape[1]
    assert(isinstance(max_features_to_correlate, int))
    corr_matrix = omics.iloc[:,0:max_features_to_correlate-1].corr().abs()
    # Select upper triangle of correlation matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
    to_drop_corr = [column for column in upper.columns if any(upper[column] > min_correlation_threshold)]
    print(datetime.now(), ct, comment,  'dropping highly correlated features', len(to_drop_corr))
    cols_to_keep = [col not in to_drop_corr for col in omics.columns]
    return cols_to_keep


In [99]:
def process_omics(ct, omics_raw, clinical_data, comment):    
    dup_idx = omics_raw.duplicated()
    print(datetime.now(), ct, comment, f'will remove duplicate features {sum(dup_idx)}')
    omics_raw = omics_raw.loc[~dup_idx]
    omics_raw = omics_raw.transpose()
    print(datetime.now(), ct, comment, omics_raw.shape)
    omics_std = omics_raw.std(axis=0, skipna=True)
    idx_omics_filter_std = omics_std <= min_feature_std
    omics_raw = omics_raw.loc[:,~idx_omics_filter_std]
    scaler = MinMaxScaler()
    omics_scaled = pd.DataFrame(scaler.fit_transform(omics_raw), columns=omics_raw.columns)
    T = clinical_data.apply(calc_time_to_event, axis = 1)
    E = (clinical_data["vital_status"] == 'Dead').astype(bool)
    importances_E = mutual_info_classif(omics_scaled, E)
    feature_importances_E = pd.Series(importances_E, omics_scaled.columns).sort_values()
    drop_low_ig_E = list(feature_importances_E[feature_importances_E==0].keys())
    print(datetime.now(), ct, comment, f'IG E:will drop {len(drop_low_ig_E)} features')
    importances_T = mutual_info_classif(omics_scaled, T)
    feature_importances_T = pd.Series(importances_T, omics_scaled.columns).sort_values()
    drop_low_ig_T = list(feature_importances_T[feature_importances_T==0].keys())
    print(datetime.now(), ct, comment, f'IG T:will drop {len(drop_low_ig_T)} features')
    to_keep = [ (col not in drop_low_ig_T) and (col not in drop_low_ig_E) for col in omics_scaled.columns]
    print(datetime.now(), ct, comment, sum(to_keep),len(to_keep))
    omics = omics_scaled.loc[:, to_keep]
    print(datetime.now(), ct, comment, omics.shape)
    omics = omics.loc[:,correlated_features(ct, comment, omics)]
    print(datetime.now(), ct, comment, 'remaining features', omics.shape[1])
    return omics

In [ ]:

    
for ct in cancers:
    # Loading raw data
    print(datetime.now(), ct, 'ENTERED')

    ct_path = f'{path_prefix}{ct}'
    clinical_data = pd.read_table(ct_path+'/clinical')
    exp_data_raw = np.log2(1+pd.read_table(ct_path+'/exp'))
    methy_data_raw = pd.read_table(ct_path+'/methy')
    mirna_data_raw = np.log2(1+pd.read_table(ct_path+'/mirna'))
    T = clinical_data.apply(calc_time_to_event, axis = 1)
    valid_T_keys = list(T[~T.isna()].index)
    # Merging 
    omics_raw = pd.concat([exp_data_raw, methy_data_raw, mirna_data_raw], axis=0, ignore_index=True)
    if len(valid_T_keys) < len(T):
        print(datetime.now(), ct, 'removing invalid rows', len(T) - len(valid_T_keys))
        omics_raw = omics_raw.iloc[:,valid_T_keys]
        exp_data_raw = exp_data_raw.iloc[:,valid_T_keys]
        methy_data_raw = methy_data_raw.iloc[:,valid_T_keys]
        mirna_data_raw = mirna_data_raw.iloc[:,valid_T_keys]
        clinical_data = clinical_data.iloc[valid_T_keys,:]
    print(datetime.now(), ct, 'merged')
    if os.path.exists(f'{ct}_omics.pkl'):
        print(datetime.now(), 'skipping', 'merged', f'{ct}_omics.pkl')
    else:
        omics = process_omics(ct, omics_raw, clinical_data, 'merged')
        pickle.dump(omics, open(f'{ct}_omics.pkl', 'wb')) 
    print(datetime.now(), ct, 'exp')
    if os.path.exists(f'{ct}_exp_omics.pkl'):
        print(datetime.now(), 'skipping', 'exp', f'{ct}_exp_omics.pkl')
    else:
        exp = process_omics(ct, exp_data_raw, clinical_data, 'exp')
        pickle.dump(exp, open(f'{ct}_exp_omics.pkl', 'wb'))
    print(datetime.now(), ct, 'methy')
    if os.path.exists(f'{ct}_methy_omics.pkl'):
        print(datetime.now(), 'skipping', 'methy', f'{ct}_methy_omics.pkl')
    else:
        methy = process_omics(ct, methy_data_raw, clinical_data, 'methy')
        pickle.dump(methy, open(f'{ct}_methy_omics.pkl', 'wb'))
    print(datetime.now(), ct, 'mirna')
    if os.path.exists(f'{ct}_mirna_omics.pkl'):
        print(datetime.now(), 'skipping', 'mirna', f'{ct}_mirna_omics.pkl')
    else:
        mirna = process_omics(ct, mirna_data_raw, clinical_data, 'mirna')
        pickle.dump(mirna, open(f'{ct}_mirna_omics.pkl', 'wb'))
    pickle.dump(clinical_data, open(f'{ct}_clinical.pkl', 'wb'))
    print(datetime.now(), ct, 'DONE')


2020-12-11 20:47:06.559752 BLCA ENTERED
negative entry in x.last_contact_days_to -21 fixing it
negative entry in x.last_contact_days_to -64 fixing it
2020-12-11 20:47:09.052080 BLCA merged
2020-12-11 20:47:09.052080 skipping merged BLCA_omics.pkl
2020-12-11 20:47:09.053079 BLCA exp
2020-12-11 20:47:09.053079 skipping exp BLCA_exp_omics.pkl
2020-12-11 20:47:09.053079 BLCA methy
2020-12-11 20:47:09.053079 skipping methy BLCA_methy_omics.pkl
2020-12-11 20:47:09.054083 BLCA mirna
2020-12-11 20:47:09.055105 skipping mirna BLCA_mirna_omics.pkl
2020-12-11 20:47:09.058090 BLCA DONE
2020-12-11 20:47:09.059063 BRCA ENTERED
negative entry in x.last_contact_days_to -7 fixing it
2020-12-11 20:47:12.897789 BRCA merged
2020-12-11 20:47:12.898787 skipping merged BRCA_omics.pkl
2020-12-11 20:47:12.898787 BRCA exp
2020-12-11 20:47:12.899785 skipping exp BRCA_exp_omics.pkl
2020-12-11 20:47:12.899785 BRCA methy
2020-12-11 20:47:12.899785 skipping methy BRCA_methy_omics.pkl
2020-12-11 20:47:12.900782 BRCA 

In [4]:
path_prefix = 'C:/Users/sharony/SurvivalAnalysis/'
if os.getlogin() =='meiry':
    path_prefix = 'D:/sharon/medical_genomics_data/'
clinical_data = pd.read_table(path_prefix+ 'BLCA/clinical')

exp_data_raw = np.log2(1+pd.read_table(path_prefix+ 'BLCA/exp'))
methy_data_raw = pd.read_table(path_prefix+ 'BLCA/methy')
mirna_data_raw = np.log2(1+pd.read_table(path_prefix+ 'BLCA/mirna'))
omics_raw = pd.concat([exp_data_raw, methy_data_raw, mirna_data_raw], axis=0, ignore_index=True)
dup_idx = omics_raw.duplicated()
print(f'will remove duplicate features{sum(dup_idx)}')
omics_raw = omics_raw.loc[~dup_idx]
omics_raw = omics_raw.transpose()
print(omics_raw.shape)

In [ ]:

T = clinical_data.apply(calc_time_to_event, axis = 1).astype('int32')
E = (clinical_data["vital_status"] == 'Dead').astype(bool)


In [3]:
max_features_to_correlate = 5000 #50000
min_correlation_threshold = 0.7


In [5]:

omics_std = omics_raw.std(axis=0, skipna=True)
print('before filtering low std\n',omics_std.describe())
print(f'will filer those with std below {min_feature_std}')
idx_omics_filter_std = omics_std <= min_feature_std # boolean vector describing std == 0
print('original features', omics_raw.shape[1])
print(f'# of features to remove (std<={min_feature_std }):', sum(idx_omics_filter_std))
omics_std = omics_raw.std(axis=0, skipna=True)
print('after filtering low std\n',omics_std.describe())
print(omics_raw.shape)

before filtering low std
 count    40258.000000
mean         0.679650
std          0.635853
min          0.000000
25%          0.249385
50%          0.321139
75%          0.917353
max          5.262542
dtype: float64
will filer those with std below 0.0
original features 40258
# of features to remove (std<=0.0): 1
after filtering low std
 count    40257.000000
mean         0.679667
std          0.635852
min          0.003800
25%          0.249389
50%          0.321146
75%          0.917365
max          5.262542
dtype: float64
(304, 40257)


In [6]:
scaler = MinMaxScaler()
omics_scaled = pd.DataFrame(scaler.fit_transform(omics_raw), columns=omics_raw.columns)

print(omics_scaled.shape)

(304, 40257)


In [7]:
def calc_time_to_event(x):
    #print(type(x) , x)
    assert x.vital_status in ['Dead', 'Alive']
    if x.vital_status=='Dead':
        return x.death_days_to
    return x.last_contact_days_to

T = clinical_data.apply(calc_time_to_event, axis = 1).astype('int32')
E = (clinical_data["vital_status"] == 'Dead').astype(bool)


In [ ]:
# entropy_T = entropy(T.value_counts(dropna=True, normalize=True).values)
# entropy_E = entropy(E.value_counts(dropna=True, normalize=True).values)
# print(entropy_E, entropy_T)
# min_ig_E_threshold = 0.1 * entropy_E
# min_ig_T_threshold = 0.1 * entropy_T
# print(min_ig_E_threshold, min_ig_T_threshold)

In [8]:
importances_E = mutual_info_classif(omics_scaled, E)
feature_importances_E = pd.Series(importances_E, omics_scaled.columns).sort_values()
drop_low_ig_E = list(feature_importances_E[feature_importances_E==0].keys())
print(f'IG E:will drop {len(drop_low_ig_E)} features')

IG E:will drop 18670 features


In [9]:
importances_T = mutual_info_classif(omics_scaled, T)
feature_importances_T = pd.Series(importances_T, omics_scaled.columns).sort_values()
drop_low_ig_T = list(feature_importances_T[feature_importances_T==0].keys())
print(f'IG T:will drop {len(drop_low_ig_T)} features')

IG T:will drop 22460 features


In [11]:
to_keep = [ (col not in drop_low_ig_T) and (col not in drop_low_ig_E) for col in omics_scaled.columns]
print(sum(to_keep),len(to_keep))

9780 40257


In [12]:
omics = omics_scaled.loc[:, to_keep]
omics.shape

(304, 9780)

In [38]:
# idx_omics_filter_std = omics_std <= min_feature_std # boolean vector describing std == 0
# print('original features', omics.shape[1])
# print(f'# of features to remove (std<={min_feature_std }):', sum(idx_omics_filter_std))
# omics = omics.loc[:,~idx_omics_filter_std]
# omics_std = omics.std(axis=0, skipna=True)
# print('remaining features', omics.shape[1])
# print(omics_std.describe())

original features 21568
# of features to remove (std<=0.1): 654
remaining features 20914
count    20914.000000
mean         0.223481
std          0.063968
min          0.100153
25%          0.163123
50%          0.241957
75%          0.279063
max          0.449797
dtype: float64


In [13]:
# Create correlation matrix
max_features_to_correlate = omics.shape[1]
min_correlation_threshold = 0.9
corr_matrix = omics.iloc[:,0:max_features_to_correlate-1].corr().abs()
# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
# Find index of feature columns with correlation greater than 0.95

In [14]:
to_drop_corr = [column for column in upper.columns if any(upper[column] > min_correlation_threshold)]
print(datetime.now(), 'dropping highly correlated features', len(to_drop_corr))
cols_to_keep = [col not in to_drop_corr for col in omics.columns]

2020-12-10 16:56:01.057313 dropping highly correlated features 391


In [15]:

pickle.dump(upper, open('upper.pkl', 'wb'))

In [114]:
upper = pickle.load(open("upper.pkl", 'rb'))

In [16]:
omics.shape

(304, 9780)

In [17]:
omics = omics.loc[:,cols_to_keep]
print('remaining features', omics.shape[1])

remaining features 9389


In [58]:
pickle.dump(omics, open('omics.pkl', 'wb'))

In [63]:
omics = pickle.load(open("omics.pkl", 'rb'))

In [19]:
from datetime import datetime
print(datetime.now())

2020-12-10 16:56:22.015589


In [57]:
import numpy as np
import matplotlib.pyplot as plt

# For preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper 
import torch # For building the networks 
import torchtuples as tt # Some useful functions
from pycox.datasets import metabric
from pycox.models import LogisticHazard
# from pycox.models import PMF
# from pycox.models import DeepHitSingle
from pycox.evaluation import EvalSurv

In [51]:

data = omics.copy()
data.loc[:,'time_to_event'] = clinical_data.apply(calc_time_to_event, axis = 1).astype('int32').values
data.loc[:,'event'] = (clinical_data['vital_status'].values == 'Dead')

data.replace([np.inf, -np.inf], np.nan)
print('null values in data', data.isnull().sum().sum())

0

In [56]:
cph = CoxPHFitter(penalizer = 0.01)
cph.fit(data, duration_col='time_to_event', event_col='event', show_progress = True)
#cph.print_summary(columns=data.columns[0:10])


Iteration 1: norm_delta = 5.15222, step_size = 0.9000, log_lik = -382.58465, newton_decrement = 578.56939, seconds_since_start = 1029.2
Iteration 2: norm_delta = 1.11734, step_size = 0.2250, log_lik = -250.65196, newton_decrement = 81.32051, seconds_since_start = 1897.2
Iteration 3: norm_delta = 0.85129, step_size = 0.2925, log_lik = -209.85963, newton_decrement = 55.62084, seconds_since_start = 2465.4
Iteration 4: norm_delta = 2.54494, step_size = 0.4943, log_lik = -171.39592, newton_decrement = 110.97268, seconds_since_start = 3007.8
Iteration 5: norm_delta = 2.05560, step_size = 0.6298, log_lik = -135.14333, newton_decrement = 34.36561, seconds_since_start = 3538.1
Iteration 6: norm_delta = 11.05935, step_size = 0.8023, log_lik = -104.47644, newton_decrement = 342.70510, seconds_since_start = 4060.7
Iteration 7: norm_delta = 2.97630, step_size = 0.2205, log_lik = -102.94604, newton_decrement = 27.82815, seconds_since_start = 4598.4
Iteration 8: norm_delta = 2.25695, step_size = 0.28

<lifelines.CoxPHFitter: fitted with 304 total observations, 224 right-censored observations>

#save to file

import pickle

with open('cph.pickle', 'wb') as handle:
    pickle.dump(cph, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('cph.pickle', 'rb') as handle:
    cph = pickle.load(handle)


In [59]:
cph.check_assumptions(data, p_value_threshold=0.05, show_plots=True)

Proportional hazard assumption looks okay.


[]

In [62]:
# Cross Validation
cox_scores= {}

from lifelines.utils import k_fold_cross_validation
scores = k_fold_cross_validation(cph, data, duration_col='time_to_event', event_col='event', k=2, scoring_method="concordance_index")
print(scores)
cox_scores['cox_regular'] = {}
cox_scores['cox_regular'][cph]= scores

ConvergenceError: delta contains nan value(s). Convergence halted. Please see the following tips in the lifelines documentation: https://lifelines.readthedocs.io/en/latest/Examples.html#problems-with-convergence-in-the-cox-proportional-hazard-model

In [ ]:
cox_scores

# Cox with Regularizations

In [ ]:
df_y = data[[ 'event','time_to_event']]

y = df_y.to_numpy(copy=True)

X = data.drop(columns=['event', 'time_to_event']).to_numpy()


In [ ]:
y2=([tuple(y[i,:]) for i in range(y.shape[0])])

y2 = np.asarray([tuple(y[i,:]) for i in range(y.shape[0])],dtype=[('e.tdm', '?'), ('t.tdm', '<f8')])

In [ ]:
# Cox 
cox = CoxnetSurvivalAnalysis(l1_ratio=1.0, alpha_min_ratio=0.00, n_alphas = 1)
cox.fit(X, y2)

In [ ]:
alphas = cox.alphas_
cv = KFold(n_splits=3, shuffle=True, random_state=0)
gcv = GridSearchCV(
    make_pipeline(cox),
    param_grid={"coxnetsurvivalanalysis__alphas": [[v] for v in alphas]},
    cv=cv,
    error_score=0.5,
    n_jobs=4).fit(X, y2)

cv_results_cox = pd.DataFrame(gcv.cv_results_)

In [ ]:
score_cox_dict = dict(zip(cv_results_cox.param_coxnetsurvivalanalysis__alphas,cv_results_cox.mean_test_score))

In [ ]:
# Ridge
cox_ridge = CoxnetSurvivalAnalysis(l1_ratio=0.0, alpha_min_ratio=0.01,n_alphas = 50)
cox_ridge.fit(X, y2)

In [ ]:
alphas = cox_ridge.alphas_
cv = KFold(n_splits=3, shuffle=True, random_state=0)
gcv = GridSearchCV(
    make_pipeline(cox_ridge),
    param_grid={"coxnetsurvivalanalysis__alphas": [[v] for v in alphas]},
    cv=cv,
    error_score=0.5,
    n_jobs=4).fit(X, y2)

cv_results_ridge = pd.DataFrame(gcv.cv_results_)

In [ ]:
score_cox_ridge_dict = dict(zip(cv_results_ridge.param_coxnetsurvivalanalysis__alphas,cv_results_ridge.mean_test_score))

In [ ]:
# Lasso
cox_lasso = CoxnetSurvivalAnalysis(l1_ratio=1.0, alpha_min_ratio=0.01,n_alphas = 50)
cox_lasso.fit(X, y2)

In [ ]:
alphas = cox_lasso.alphas_
cv = KFold(n_splits=3, shuffle=True, random_state=0)
gcv = GridSearchCV(
    make_pipeline(cox_lasso),
    param_grid={"coxnetsurvivalanalysis__alphas": [[v] for v in alphas]},
    cv=cv,
    error_score=0.5,
    n_jobs=4).fit(X, y2)

cv_results_lasso = pd.DataFrame(gcv.cv_results_)

In [ ]:
score_cox_lasso_dict = dict(zip(cv_results_lasso.param_coxnetsurvivalanalysis__alphas,cv_results_lasso.mean_test_score))

In [ ]:
# Elastic Net
cox_elastic_net = CoxnetSurvivalAnalysis(l1_ratio=0.5, alpha_min_ratio=0.01, n_alphas =50)
cox_elastic_net.fit(X, y2)

In [ ]:
alphas = cox_elastic_net.alphas_
cv = KFold(n_splits=3, shuffle=True, random_state=0)
gcv = GridSearchCV(
    make_pipeline(cox_elastic_net),
    param_grid={"coxnetsurvivalanalysis__alphas": [[v] for v in alphas]},
    cv=cv,
    error_score=0.5,
    n_jobs=4).fit(X, y2)

cv_results_elastic_net = pd.DataFrame(gcv.cv_results_)

In [ ]:
score_elastic_net_dict = dict(zip(cv_results_elastic_net.param_coxnetsurvivalanalysis__alphas,cv_results_elastic_net.mean_test_score))